# Tucker Decomposition on CIFAR-10

## 1. Phương pháp

### Phương pháp 1:Tucker-2 Decomposition

**Ý tưởng:**
Chỉ phân rã theo hai mode kênh (input channels và output channels) — thường dùng nhất để nén convolution kernels trong CNN: giảm số channel đầu vào/đầu ra, giữ nguyên không gian (k×k).

**Công thức**
$$
W \in \mathbb{R}^{k \times k \times C_{in} \times C_{out}} \approx
G \times_3 U^{(3)} \times_4 U^{(4)}
$$
Trong đó:
$G \in \mathbb{R}^{k \times k \times r_{in} \times r_{out}}$,
$U^{(3)} \in \mathbb{R}^{C_{in} \times r_{in}}$, $U^{(4)} \in \mathbb{R}^{C_{out} \times r_{out}}$.

**Thuật toán / Triển khai (CNN):**

1.Tính HOSVD/HOOI cho mode-3 và mode-4 để lấy $U^{(3)},U^{(4)}$ và core $G$.

2.Thay 1 conv ($k\times k$, $C_{in}\to C_{out}$) bằng chuỗi:

    1×1 conv: giảm $C_{in}\to r_{in}$ (dùng $U^{(3)}$),

    k×k conv: $r_{in}\to r_{out}$ (dùng core $G$),

    1×1 conv: tăng $r_{out}\to C_{out}$ (dùng $U^{(4)}$).

3.Fine-tune toàn mạng.


### Phương pháp 2:Full Tucker Decomposition

**Ý tưởng:**
Phân rã tensor trọng số theo cả 4 mode (spatial + input channel + output channel). Nén mạnh nhưng phức tạp hơn.
**Công thức**
$$
W \in \mathbb{R}^{k \times k \times C_{in} \times C_{out}} \approx
G \times_1 U^{(1)} \times_2 U^{(2)} \times_3 U^{(3)} \times_4 U^{(4)}
$$

Trong đó:
$U^{(1)}\in\mathbb{R}^{k\times r_1}, U^{(2)}\in\mathbb{R}^{k\times r_2}, U^{(3)}\in\mathbb{R}^{C_{in}\times r_3}, U^{(4)}\in\mathbb{R}^{C_{out}\times r_4}$ và $G\in\mathbb{R}^{r_1\times r_2\times r_3\times r_4}$.

**Thuật toán / Triển khai:**

Chọn ranks $(r_1,r_2,r_3,r_4)$.

Tính HOSVD/HOOI để thu $U^{(i)}$ và core $G$.

Thay conv gốc bằng chuỗi layer tương ứng (có thể chuyển các factor thành conv 1D/1×k/ k×1/1×1 tùy thiết kế).

Fine-tune.

### Phương pháp 3:Asymmetric Tucker Decomposition

**Ý tưởng:**

Trong Tucker truyền thống (Tucker-2 hoặc Full Tucker), chúng ta thường giảm cả input channels và output channels đối xứng.

Nhưng thực tế, CNN layers thường có số lượng input và output channel khác nhau, và mức độ dư thừa cũng khác nhau.

Asymmetric Tucker cho phép chỉ giảm ở một phía (chỉ input hoặc output) → linh hoạt hơn, ít ảnh hưởng đến accuracy.

**Công thức(Nếu giảm input channels)**
$$
W \in \mathbb{R}^{k \times k \times C_{in} \times C_{out}}
\;\approx\;
G \times_3 U^{(3)}
$$

Trong đó:

$U^{(3)} \in \mathbb{R}^{C_{in} \times r_{in}}$ nén số chiều input từ $C_{in}$ xuống $r_{in}$.

$G \in \mathbb{R}^{k \times k \times r_{in} \times C_{out}}$ là core tensor.

**Công thức(Nếu giảm output channels)**
$$
W \;\approx\; G \times_4 U^{(4)}, \quad U^{(4)} \in \mathbb{R}^{C_{out} \times r_{out}}
$$

T**riển khai trong CNN:**

Nếu giảm input → thay conv layer bằng 1×1 conv (giảm $C_{in} \to r_{in}$) rồi đến k×k conv ($r_{in} \to C_{out}$).

Nếu giảm output → thay conv layer bằng k×k conv ($C_{in} \to r_{out}$) rồi đến 1×1 conv ($r_{out} \to C_{out}$).

### Phương pháp 4:Data-driven Tucker (Adaptive Tucker)

**Ý tưởng:**

Thay vì decomposition hoàn toàn dựa vào trọng số $W$ hiện tại, ta dùng dữ liệu huấn luyện để tìm decomposition tối ưu sao cho loss trên dữ liệu gốc nhỏ nhất.

Tức là decomposition được học end-to-end với dữ liệu, chứ không chỉ là phân tích tuyến tính.

→ Hướng nâng cao, kết hợp compression với training.

**Công thức**
$$
\min_{G, \{U^{(i)}\}} \;
\mathbb{E}_{(x,y)\sim \mathcal{D}}
\Big[ \mathcal{L}\big(f_{G,U}(x), y\big) \Big]
$$

Trong đó:

$f_{G,U}$ là mạng sau khi thay các layer gốc bằng Tucker decomposition với core $G$ và factor ${U^{(i)}}$.

Ta tối ưu trực tiếp $G$ và ${U^{(i)}}$ bằng gradient descent, thay vì tính bằng HOSVD/HOOI như Tucker thường.

**Triển khai trong CNN:**

Khởi tạo decomposition bằng Tucker-2 hoặc Tucker đầy đủ.

Chèn vào mạng → coi core $G$ và factor $U^{(i)}$ như tham số learnable.

Train/fine-tune lại trên dữ liệu gốc → decomposition thích nghi với dữ liệu.

# 2.Đánh giá